In [1]:
import pandas as pd
import spacy 
from tqdm import tqdm
import csv
from spacy.matcher import PhraseMatcher
from sklearn.utils import shuffle
nlp = spacy.load("en_core_web_lg")

In [3]:
#no stopwords contains NAN sentences
columnname = 'sentence'
sentences_jobpostings = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/sentences/sentences_emscad.csv")
trigrams = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Combined/Taxonomy/n-grams.csv",sep=',')
trigrams = trigrams[['allgrams']]

In [4]:
matcher = PhraseMatcher(nlp.vocab)

#Adding all the trigrams to the matcher
for gram in tqdm(trigrams['allgrams']):
    matcher.add("NGRAM", [nlp(gram)])


100%|██████████| 8985/8985 [00:48<00:00, 184.07it/s]


In [5]:
leftcontextstorage = []
recoveredgramstorage = []
rightcontextstorage = []

for sentence in tqdm(sentences_jobpostings[columnname]):

    #Intiatilizing the sentence and searching for matches
    doc = nlp(sentence)
    matches = matcher(doc)

    for match_id, start, end in matches:
        
        #Getting up to 5 contextwords next to the grams
        for contextnumbers in range(1,6,1):

            #Defining the number of context words
            leftcontextnumbers = contextnumbers
            rightcontextnumbers = contextnumbers

            #Checking if there is any context to the left or right of the gram, if not skipping the gram
            if start == 0 or end == len(doc):
                continue

            #Checking if there is enough context to the left of the gram
            if start < leftcontextnumbers:
                leftcontextnumbers = start
            #Checking if there is enough context to the right of the gram
            elif end+rightcontextnumbers > len(doc):
                rightcontextnumbers = len(doc)-end

            # Get the left context of the gram
            leftcontext = doc[start-leftcontextnumbers:start]
            # Get the recovered gram by slicing the Doc 
            recoveredgram = doc[start:end]
            # Get the right context of the gram
            rightcontext = doc[end:(end+rightcontextnumbers)]


            leftcontextstorage.append(leftcontext.text)
            recoveredgramstorage.append(recoveredgram.text)
            rightcontextstorage.append(rightcontext.text)
            
df = pd.DataFrame(zip(leftcontextstorage,recoveredgramstorage,rightcontextstorage),
                 columns=['left_context', 'candidate_skill', 'right_context'])


100%|██████████| 302284/302284 [44:47<00:00, 112.47it/s] 


In [6]:
df = df.drop_duplicates()
df = shuffle(df)

In [7]:
#Removing any double whitespaces
df['left_context'] = df['left_context'].str.replace(r'[ ]{2,}', ' ', regex=True)
df['candidate_skill'] = df['candidate_skill'].str.replace(r'[ ]{2,}', ' ', regex=True)
df['right_context'] = df['right_context'].str.replace(r'[ ]{2,}', ' ', regex=True)

df['concatenated'] = df['left_context'] + ' | ' + df['candidate_skill'] + ' | ' + df['right_context']

In [8]:
concat = df[['concatenated']]
concat['label'] = ''
concat

<ipython-input-8-55ec7716b5c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concat['label'] = ''


,concatenated,label
14601,preferred | python php vb or | programming la...,
60468,a vacancy for a | java j2ee developer fluent |...,
70099,side components using python ruby | java scala...,
75649,our client is seeking a | java developer who |...,
2588,skills | python oracle teradata java | jee lin...,
...,...,...
337,advantage for the | successful java | develope...,
42340,be | proficient with java | and,
40470,software | engineer java will contribute | to,
22535,strong | java | coder,


In [9]:
df.to_csv("//Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Combined/Taxonomy/Sorted/context_sneaky.csv",
          quoting=csv.QUOTE_NONNUMERIC,
          index=False)